In [1]:
import setup
setup.init_django()

In [2]:
import helpers.clients as helper_clients

In [6]:
company_name = "Microsoft"
company_ticker = "MSFT"
multiplier = 1
from_date = "2023-01-09"
to_date = "2023-04-09"

In [7]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

In [8]:
len(dataset)

41616

In [25]:
import pandas as pd

# Load the JSON data into a pandas DataFrame
df = pd.json_normalize(dataset)

# Display the DataFrame
display(df)

,open_price,close_price,high_price,low_price,number_of_trades,volume,volume_weighted_average,raw_timestamp,time
0,225.16,225.16,225.16,225.16,16,588.0,225.2090,1673254800000,2023-01-09 09:00:00+00:00
1,225.51,225.60,225.60,225.51,24,1055.0,225.5642,1673254920000,2023-01-09 09:02:00+00:00
2,226.00,226.00,226.00,226.00,21,1019.0,225.9993,1673255160000,2023-01-09 09:06:00+00:00
3,226.00,226.00,226.00,226.00,13,351.0,225.9972,1673255520000,2023-01-09 09:12:00+00:00
4,225.80,225.75,225.80,225.75,19,1742.0,225.7828,1673255640000,2023-01-09 09:14:00+00:00
...,...,...,...,...,...,...,...,...,...
41611,291.60,291.75,291.75,291.60,16,711.0,291.6942,1680825300000,2023-04-06 23:55:00+00:00
41612,291.75,291.60,291.75,291.60,18,393.0,291.6862,1680825360000,2023-04-06 23:56:00+00:00
41613,291.86,291.62,291.86,291.62,26,729.0,291.7001,1680825420000,2023-04-06 23:57:00+00:00
41614,291.60,291.60,291.60,291.60,8,336.0,291.6076,1680825480000,2023-04-06 23:58:00+00:00


In [27]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 225.16,
 'close_price': 225.16,
 'high_price': 225.16,
 'low_price': 225.16,
 'number_of_trades': 16,
 'volume': 588,
 'volume_weighted_average': 225.209,
 'raw_timestamp': 1673254800000,
 'time': datetime.datetime(2023, 1, 9, 9, 0, tzinfo=<UTC>)}

In [18]:
from market.models import Company, StockQuote

In [19]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [28]:
company_obj

<Company: Microsoft (MSFT)>

In [29]:
StockQuote.objects.create(company=company_obj, **new_stock_data)

#StockQuote.objects.bulk_create(company=company_obj, **new_stock_data)

IntegrityError: duplicate key value violates unique constraint "1_2_market_stockquote_company_id_time_f15d280e_uniq"
DETAIL:  Key (company_id, "time")=(3, 2023-01-09 09:00:00+00) already exists.

In [46]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )

In [24]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>, <StockQuote: StockQuote object (3)>, <StockQuote: StockQuote object (4)>, <StockQuote: StockQuote object (5)>, <StockQuote: StockQuote object (6)>, <StockQuote: StockQuote object (7)>, <StockQuote: StockQuote object (8)>, <StockQuote: StockQuote object (9)>, <StockQuote: StockQuote object (10)>, <StockQuote: StockQuote object (11)>, <StockQuote: StockQuote object (12)>, <StockQuote: StockQuote object (13)>, <StockQuote: StockQuote object (14)>, <StockQuote: StockQuote object (15)>, <StockQuote: StockQuote object (16)>, <StockQuote: StockQuote object (17)>, <StockQuote: StockQuote object (18)>, <StockQuote: StockQuote object (19)>, <StockQuote: StockQuote object (20)>, <StockQuote: StockQuote object (21)>, '...(remaining elements truncated)...']>

In [14]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
        
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [15]:
StockQuote.objects.all().count()

41618

In [35]:
dataset[0]

{'open_price': 225.16,
 'close_price': 225.16,
 'high_price': 225.16,
 'low_price': 225.16,
 'number_of_trades': 16,
 'volume': 588,
 'volume_weighted_average': 225.209,
 'raw_timestamp': 1673254800000,
 'time': datetime.datetime(2023, 1, 9, 9, 0, tzinfo=<UTC>)}

In [36]:
len(dataset[0])

9

In [37]:
len(dataset)

41616

In [44]:
batch_size = 2

range(0, len(dataset[0]), batch_size)

range(0, 9, 2)

In [52]:
#dataset[0:3]

In [54]:
chunked_quotes = []

for data in dataset[0:2]:
    chunked_quotes.append(StockQuote(company=company_obj, **data))

chunked_quotes

[<StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>]

In [49]:
batch_size = 2
for i in range(0, len(dataset[0]), batch_size):
    print(i)
    batch_chunk = dataset[i:i+batch_size]
    print(batch_chunk)
    

0
[{'open_price': 225.16, 'close_price': 225.16, 'high_price': 225.16, 'low_price': 225.16, 'number_of_trades': 16, 'volume': 588, 'volume_weighted_average': 225.209, 'raw_timestamp': 1673254800000, 'time': datetime.datetime(2023, 1, 9, 9, 0, tzinfo=<UTC>)}, {'open_price': 225.51, 'close_price': 225.6, 'high_price': 225.6, 'low_price': 225.51, 'number_of_trades': 24, 'volume': 1055, 'volume_weighted_average': 225.5642, 'raw_timestamp': 1673254920000, 'time': datetime.datetime(2023, 1, 9, 9, 2, tzinfo=<UTC>)}]
2
[{'open_price': 226, 'close_price': 226, 'high_price': 226, 'low_price': 226, 'number_of_trades': 21, 'volume': 1019, 'volume_weighted_average': 225.9993, 'raw_timestamp': 1673255160000, 'time': datetime.datetime(2023, 1, 9, 9, 6, tzinfo=<UTC>)}, {'open_price': 226, 'close_price': 226, 'high_price': 226, 'low_price': 226, 'number_of_trades': 13, 'volume': 351, 'volume_weighted_average': 225.9972, 'raw_timestamp': 1673255520000, 'time': datetime.datetime(2023, 1, 9, 9, 12, tzinfo

In [39]:
batch_size = 1

#range(0, len(dataset[0]), batch_size)
range(0, len(dataset), batch_size)

range(0, 41616)

In [34]:
batch_size = 1
for i in range(0, len(dataset), batch_size):
    print(i)

0
1
2
3
4
5
6
7
8
